In [1]:
#Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install langchain openai lark tiktoken faiss numpy gym==0.25.1 --upgrade

In [6]:
import os, json
os.environ['JAVA_HOME'] = "/home/yh/anaconda3/envs/webshop/bin/"
os.environ['JVM_PATH'] = "/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so"
# Open the file and load the JSON data
with open("./.api_keys.txt", "r") as file:
    api_keys = json.load(file)

os.environ["OPENAI_API_KEY"] = api_keys["OPENAI_API_KEY"]
#os.environ["OPENAI_API_KEY"]

In [3]:
import os

# Change to a directory
os.chdir('/home/yh/LLMAgent/webshop/baseline_models')


from train_rl import parse_args as webenv_args
import imp
from env import WebEnv 


args = webenv_args()[0]
env = WebEnv(args, split='test')
print('env loaded')

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/tmp/ipykernel_2358/1303183564.py:8: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


Products loaded.

Keys cleaned.

Attributes loaded.

100%|██████████████████████████████████████████████████████████████████████| 1181436/1181436 [00:14<00:00, 83923.88it/s]


164 skipped

Loaded 12087 goals.
Total huamn goals considered： range(0, 500)


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


env loaded


In [4]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

from autogpt import AutoGPT
from langchain.chat_models import ChatOpenAI
from output_parser import (
    AutoGPTOutputParser,
    BaseAutoGPTOutputParser,
)
from prompt import AutoGPTPrompt
from langchain.chains.llm import LLMChain

def convert_info(info: dict) -> dict:
    actions = []
    for a in info['valid']:
        if a.startswith('click[item - '):
            actions.append(f'click[{env.name2asin[a[13:-1]]}]')
        else:
            actions.append(a)
    info['valid'] = actions
    return info

def search(action: str) -> str:
    global env
    obs, reward, done, info = env.step(f"search[{action}]")
    valid_actions = convert_info(info)['valid']
    assert len(valid_actions) > 0
    return "Observation:\n" + obs + "Valid Actions:\n" + str(valid_actions)

def click(action: str) -> str:
    global env
    obs, reward, done, info = env.step(f"click[{action}]")
    obs_prompt = "Observation:\n" + obs
    valid_actions = convert_info(info)['valid']
    if valid_actions[0].startswith("search["):
        return obs_prompt + "\n:Valid Actions:\n['search[<QUERY>]']"
    else:
        action_prompt = "\nValid Actions:\n" + str(valid_actions)
    if done: 
        return "{'reward':%s}" % reward    
    return obs_prompt + action_prompt

def process_result(full_msg_log):
    import re
    # print(reward_msg)
    output_parser = AutoGPTOutputParser()

    parsed_action = output_parser.parse(full_msg_log[-1].content)
    if parsed_action.name == "finish":
        reward_msg = full_msg_log[-4].content
        match = re.search(r"'reward':([\d\.]+)", reward_msg)
        if match:
            reward = float(match.group(1))
        else:
            reward = 0.0
        return {"purchase": True, "reward": reward * 10}
    else:
        return {"purchase": False, "reward": 0.0}

def init_vectorstore(embedding_size=1536):
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})
    return vectorstore

In [5]:
webshop_search_tool = Tool(
    name = "search",
    func=search,
    description="""
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output in the format as search[<query>].
        For example, for instruction 'i need a long clip-in hair extension which is natural looking,
        and price lower than 30.00 dollars', the query could be like
        'long clip-in hair extension natural looking, lower than 30.00 dollars'
    """
)

webshop_click_tool = Tool(
    name = "click",
    func = click,
    description="""
        Requirement:
        1. You should strictly limit the choises of your actions from the Valid Action list, all in 'click[<input>]' format.
        2. In output JSON, use "click" for "command.name", and use the variable <input> for "command.args.tool_input".  
        3. Please make the purchase if you are running out of actions, indicated in 'loop number:'.
        Example: 
        1. The valid actions has'click[b09mm9ncqq]', you use 'click' as "command.name" and 'b09mm9ncqq' as "command.args.tool_input". 
         The actual item description for 'b09mm9ncqq' can be found in the Observation section. 
        2. The valid actions has 'click[back to search]', you use 'click' as "command.name" and 'back to search' as "command.args.tool_input".
        3. The valid actions has 'click[buy now]', you use click as "command.name" and "buy now" as "command.args.tool_input".
    """
)

tools = [
    webshop_search_tool,
    webshop_click_tool
]

In [6]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss




In [7]:
ai_name = "Shopping AI"
ai_role = "Web Shopper"
llm = ChatOpenAI(temperature=0)



In [8]:
LOOP_LIMIT = 20
input_variables=["memory", "messages", "goals", "user_input"]

def init_agent(ai_name, ai_role, tools, input_variables, llm, vectorstore, verbose = True):
    prompt = AutoGPTPrompt(
                ai_name=ai_name,
                ai_role=ai_role,
                tools=tools,
                input_variables=input_variables,
                token_counter=llm.get_num_tokens,
            )
    chain = LLMChain(llm=llm, prompt=prompt)


    agent = AutoGPT(
        ai_name="Shopping AI",
        memory=vectorstore.as_retriever(),
        chain=chain,
        output_parser=AutoGPTOutputParser(),
        tools=tools,
        loop_limit=LOOP_LIMIT
    )
    agent.chain.verbose = verbose
    return agent




In [9]:
results = []
full_logs = []
for i in range(0,20):
    obs, info = env.reset(i)
    print(f"############### Goal {i}: {info['goal']} ###############")
    agent = init_agent(ai_name, ai_role, tools, input_variables, llm, init_vectorstore(), verbose=False)

    full_msg_log, actions = agent.run([f"I want you to help me find the right item and make the purchase in {LOOP_LIMIT} actions. " + info['goal']])
    results.append(process_result(full_msg_log))
    full_logs.append(full_msg_log)
    


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


############### Goal 0: Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 40.00 dollars ###############
{
    "thoughts": {
        "text": "I will start by searching for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 40.00 dollars",
        "reasoning": "I need to find the right item that meets all the requirements specified in the instruction. I will start by searching for the item and then click on the item to purchase it.",
        "plan": "- Search for the item using the best query\n- Click on the item to purchase it",
        "criticism": "I need to be careful with my search query to ensure that I find the right item that meets all the requirements."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "gluten free, 100% vegan plant based protein shake soy-free, price lower than 40.00 dollars"
        }
    }
}
Command search 

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


{
    "thoughts": {
        "text": "I will click on the first item that meets all the requirements specified in the instruction.",
        "reasoning": "I need to click on the item that meets all the requirements specified in the instruction to purchase it.",
        "plan": "- Click on the item that meets all the requirements specified in the instruction to purchase it.",
        "criticism": "I need to be careful to ensure that I click on the right item that meets all the requirements specified in the instruction."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "b07fypsnh8"
        }
    }
}
Command click returned: Observation:
Instruction:
i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 40.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
OWYN - 100% Vegan Plant-Based Protein Shakes | Cold Brew Coffee, 12 Fl Oz | Dairy-Free, Gluten-Free, Soy-Free, Tree Nut-

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' i

...

{
    "thoughts": {
        "text": "I will click on the second item that meets the criteria.",
        "reasoning": "The second item that meets the criteria is the BAYCHEER Tiffany Style Stained Glass Bowl Shade Pendant Light Chandelier Decorative Hanging Lamp Pendant Lighting Adjustable Ceiling Fixture with Pull Chain 3 Lights for Living Room Dining Room.",
        "plan": "- search for pendant lights for ceiling lights for living room, and price lower than 80.00 dollars\n- click on the first item that meets the criteria\n- click on the second item that meets the criteria\n- purchase the item",
        "criticism": "I need to be careful with my search query to ensure that I find the right item."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "b0854jtzmt"
        }
    }
}
Command click returned: Observation:
Instruction:
i'm looking for the pendant lights for ceiling lights for living room, and price lower than 80.00 dollars
[button] Back

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' i

{
    "thoughts": {
        "text": "I will purchase the item.",
        "reasoning": "The item meets the criteria and is within my budget, so I will purchase it.",
        "plan": "- search for pendant lights for ceiling lights for living room, and price lower than 80.00 dollars\n- click on the first item that meets the criteria\n- click on the second item that meets the criteria\n- purchase the item",
        "criticism": "I need to be careful with my search query to ensure that I find the right item."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "buy now"
        }
    }
}
Command click returned: {'reward':3.333333333333333}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


{
    "thoughts": {
        "text": "I will finish my objectives.",
        "reasoning": "I have found and purchased the item that meets the criteria, so I will finish my objectives.",
        "plan": "- search for pendant lights for ceiling lights for living room, and price lower than 80.00 dollars\n- click on the first item that meets the criteria\n- click on the second item that meets the criteria\n- purchase the item",
        "criticism": "I need to be careful with my search query to ensure that I find the right item."
    },
    "command": {
        "name": "finish",
        "args": {
            "response": "I have completed my objectives."
        }
    }
}


In [10]:
import pandas as pd
import numpy as np
# Convert the dictionary to a DataFrame
df = pd.DataFrame(results, dtype = np.float64)

df_string = df.to_csv(sep='\t', index=False)
print(df_string)

purchase	reward
1.0	100.0
1.0	33.33333333333333
0.0	0.0
1.0	66.66666666666666
1.0	66.66666666666666
1.0	50.0
1.0	75.0
1.0	100.0
1.0	0.0
1.0	66.66666666666666
1.0	66.66666666666666
0.0	0.0
1.0	0.0
1.0	33.33333333333333
1.0	0.0
1.0	0.0
1.0	66.66666666666666
1.0	12.5
1.0	25.0
1.0	33.33333333333333



In [11]:
import json

def write_dict_to_file(dictionary, file_path):
    # Open the file in write mode
    with open(file_path, "w") as file:
        for line in dictionary:
            file.write(str(line) + "\n")

# Example usage
file_path = "full_logs_test_item_0_to_19.txt"
write_dict_to_file(full_logs, file_path)

In [29]:
print(full_logs)

[[HumanMessage(content='Determine which next command to use, and respond using the format specified above:', additional_kwargs={}, example=False), AIMessage(content='{\n    "thoughts": {\n        "text": "I will start by searching for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 40.00 dollars",\n        "reasoning": "I need to find the right item that meets the requirements given by the user",\n        "plan": "- Search for the item using the given requirements\\n- Click on the item if it meets the requirements\\n- If the item is not found, go back to search and try again\\n- If the item is found, click on buy now to make the purchase",\n        "criticism": "I need to be efficient with my searches to ensure I find the right item within the given number of actions"\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "tool_input": "gluten free, 100% vegan plant based protein shake soy-free, price lower than 4

In [15]:
print(full_logs[-1])

[HumanMessage(content='Determine which next command to use, and respond using the format specified above:', additional_kwargs={}, example=False), AIMessage(content='{\n    "thoughts": {\n        "text": "I will start by searching for \'pink niuta 2 pack hair towel wrap for dry hair\' and see if I can find it within the price range of 40.00 dollars.",\n        "reasoning": "I will start with a specific search query to narrow down the results and then adjust the query if necessary.",\n        "plan": "- Search for \'pink niuta 2 pack hair towel wrap for dry hair\'\\n- Check if the price is within the range of 40.00 dollars\\n- If not, adjust the search query\\n- Click on the item to view details\\n- If it meets the criteria, click on \'buy now\'",\n        "criticism": "I need to be careful not to waste too many actions on searching and make sure to check the price before clicking on the item."\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "tool_inp

In [20]:
for i in range(10, 20):
    obs, info = env.reset(i)
    print(f"############### Goal {i}: {info['goal']} ###############")
    agent = init_agent(ai_name, ai_role, tools, input_variables, llm, init_vectorstore())

    full_msg_log, actions = agent.run([f"I want you to help me find the right item and make the purchase in {LOOP_LIMIT} actions. " + info['goal']])
    results.append(process_result(full_msg_log))
    full_logs.append(full_msg_log)
    print(f"##############################")
    

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


############### Goal 10: Instruction: i am looking fresh scent body lotion for dry skin, and price lower than 30.00 dollars ###############


> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking fresh scent body lotion for dry skin, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query 

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking fresh scent body lotion for dry skin, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output in the format as search[<query>].
   


> Finished chain.
{
    "thoughts": {
        "text": "I should click on the first item that meets the criteria",
        "reasoning": "The first item that meets the criteria is the best option to choose from. This will save me time and actions.",
        "plan": "- Click on the first item that meets the criteria",
        "criticism": "I need to be careful to ensure that the item meets the criteria before clicking on it."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "b087y28xlm"
        }
    }
}


...



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i'm looking for the pendant lights for ceiling lights for living room, and price lower than 90.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output in the format as 


> Finished chain.
I am unable to determine the next command as I do not have access to the webpage. Please provide me with the available options.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i'm looking for the pendant lights for ceiling lights for living room, and price lower than 90.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax cons


> Finished chain.
I am unable to determine the next command as I do not have access to the webpage. Please provide me with the available options.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i'm looking for the pendant lights for ceiling lights for living room, and price lower than 90.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax cons


> Finished chain.
{
    "thoughts": {
        "text": "I need to purchase the item as it meets the criteria of being a pendant light for ceiling lights for living room with a price lower than 90.00 dollars.",
        "reasoning": "I have clicked on the item that meets the criteria given by the user and now need to purchase the item.",
        "plan": "- Search for pendant lights for ceiling lights for living room with a price lower than 90.00 dollars.\n- Click on the item that meets the criteria.\n- Purchase the item.",
        "criticism": "I need to make sure that I am purchasing the right item and not wasting any actions."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "buy now"
        }
    }
}


> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the


> Finished chain.
Great! Let's use the "finish" command to signal that we have completed our objectives. 

{
    "command": {
        "name": "finish",
        "args": {
            "response": "I have completed my objectives."
        }
    }
}
##############################


In [21]:
# Convert the dictionary to a DataFrame
df = pd.DataFrame(results, dtype = np.float64)

df_string = df.to_csv(sep='\t', index=False)
print(df_string)

purchase	reward
1.0	75.0
1.0	66.66666666666666
1.0	66.66666666666666
1.0	50.0
1.0	75.0
1.0	0.0
1.0	50.0
1.0	66.66666666666666
1.0	66.66666666666666
1.0	33.33333333333333
1.0	50.0
1.0	33.33333333333333
1.0	0.0
1.0	0.0
1.0	0.0
1.0	12.5
1.0	25.0
1.0	0.0



In [24]:
import json

def write_dict_to_file(dictionary, file_path):
    # Open the file in write mode
    with open(file_path, "w") as file:
        for line in dictionary:
            file.write(str(line) + "\n")

# Example usage
file_path = "full_logs_test_item_2_to_19.txt"
write_dict_to_file(full_logs, file_path)


# Below is For Testing Purpose

In [24]:
#obs, info = env.reset(0)
#print(info['goal'])

Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [25]:
#agent = init_agent(ai_name, ai_role, tools, input_variables, llm, init_vectorstore())

#full_msg_log, actions = agent.run([f"I want you to help me find the right item and make the purchase in {LOOP_LIMIT} actions. " + info['goal']])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output i

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output i


> Finished chain.
{
    "thoughts": {
        "text": "I will click on the first item that meets all the requirements.",
        "reasoning": "The first item that meets all the requirements is the best option to purchase.",
        "plan": "- Click on the first item that meets all the requirements.",
        "criticism": "I need to ensure that the item meets all the requirements before clicking on it."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "b07fypsnh8"
        }
    }
}


> Entering new LLMChain chain...


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' i

Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output in the format as search[<query>].
  


> Finished chain.
{
    "thoughts": {
        "text": "I will purchase the item since it meets all the requirements and is within the price range.",
        "reasoning": "The item meets all the requirements and is within the price range, so it is the best option to purchase.",
        "plan": "- Purchase the item.",
        "criticism": "I need to ensure that the item is added to the cart and the purchase is successful."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "buy now"
        }
    }
}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output i


> Finished chain.
{
    "thoughts": {
        "text": "I will finish my shopping since I have successfully purchased the item.",
        "reasoning": "Since I have successfully purchased the item, there is no need to continue shopping.",
        "plan": "- Finish my shopping.",
        "criticism": "I need to ensure that the purchase is successful and the item is delivered."
    },
    "command": {
        "name": "finish",
        "args": {
            "response": "I have successfully purchased the gluten-free, 100% vegan plant-based protein shake that is soy-free and within the price range. Shopping complete."
        }
    }
}


In [33]:
results = []
result_01 = process_result(full_msg_log)
results.append(result_01)

In [31]:
#obs, info = env.reset(1)
#print(f"############### {info['goal']} ###############")
#agent = init_agent(ai_name, ai_role, tools, input_variables, llm, init_vectorstore(), verbose = False)

#full_msg_log, actions = agent.run([f"I want you to help me find the right item and make the purchase in {LOOP_LIMIT} actions. " + info['goal']])

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


################### Instruction: i need a 120 ml hair mask treatment, and price lower than 140.00 dollars ########################
{
    "thoughts": {
        "text": "I will start by searching for a 120 ml hair mask treatment with a price lower than 140.00 dollars.",
        "reasoning": "I will use the search command to find the hair mask treatment. I will also keep track of my past search queries to avoid repetitive searches.",
        "plan": "- Use the search command to find a 120 ml hair mask treatment with a price lower than 140.00 dollars.\n- Check my past search queries to avoid repetitive searches.\n- If I cannot find the right item, I will relax the constraint on the search query.\n- If I am running out of actions, I will make the purchase.",
        "criticism": "I need to be efficient with my search queries to avoid wasting actions."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "120 ml hair mask treatment price lower than 14

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


{
    "thoughts": {
        "text": "I will click on the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "reasoning": "The Linange Milk and Keratin Mask 500ml hair mask treatment is the only item on the page that meets the requirements of being a 120 ml hair mask treatment with a price lower than 140.00 dollars.",
        "plan": "- Click on the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "criticism": "I need to be careful to ensure that I am clicking on the correct item."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B079HGJ5MH"
        }
    }
}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


I will click on the 'Buy Now' button to purchase the Linange Milk and Keratin Mask 500ml hair mask treatment.

{
    "thoughts": {
        "text": "I will click on the 'Buy Now' button to purchase the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "reasoning": "The 'Buy Now' button is the next logical step to purchase the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "plan": "- Click on the 'Buy Now' button to purchase the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "criticism": "I need to ensure that I am clicking on the correct button to make the purchase."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "buy now"
        }
    }
}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


The purchase has been made, so I will use the 'finish' command to signal that I have completed all my objectives.

{
    "thoughts": {
        "text": "I have completed the purchase of the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "reasoning": "Since the purchase has been made, there are no further actions required.",
        "plan": "- Use the 'finish' command to signal that I have completed all my objectives.",
        "criticism": "I need to ensure that I have completed all my objectives before using the 'finish' command."
    },
    "command": {
        "name": "finish",
        "args": {
            "response": "I have completed the purchase of the Linange Milk and Keratin Mask 500ml hair mask treatment."
        }
    }
}


In [38]:
print(full_msg_log)

[HumanMessage(content='Determine which next command to use, and respond using the format specified above:', additional_kwargs={}, example=False), AIMessage(content='{\n    "thoughts": {\n        "text": "I will start by searching for orange cargo pants that are suitable for everyday casual wear and hiking, and cost less than $30.",\n        "reasoning": "I need to find the right item that meets all the requirements specified by the user.",\n        "plan": "- Search for \'orange cargo pants for everyday casual wear and hiking\'\\n- Check the search results for the right item\\n- Click on the item to view more details\\n- If the item meets all the requirements, click on \'buy now\'\\n- If not, click on \'back to search\' and repeat the process",\n        "criticism": "I need to be careful not to waste too many actions on repetitive searches or irrelevant items."\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "tool_input": "orange cargo pants for eve

In [1]:
from output_parser import AutoGPTOutputParser
msg = """
I have successfully purchased the hair mask. I will now use the finish command to signal that I have completed all my objectives.

{
    "thoughts": {
        "text": "I have successfully purchased the hair mask. I will now use the finish command to signal that I have completed all my objectives.",
        "reasoning": "I have found the hair mask with the right price and size and have successfully purchased it. I will now use the finish command to signal that I have completed all my objectives.",
        "plan": "- Use the finish command to signal that I have completed all my objectives.",
        "criticism": "None"
    },
    "command": {
        "name": "finish",
        "args": {
            "response": "I have successfully purchased the hair mask and completed all my objectives."
        }
    }
}
"""
output_parser = AutoGPTOutputParser()
output_parser.parse(msg)

AutoGPTAction(name='finish', args={'response': 'I have successfully purchased the hair mask and completed all my objectives.'})

In [28]:
obs, info = env.reset(0)
action = "search[long clip-in hair extension natural looking]"
obs, reward, done, info = env.step(action)
info = convert_info(info)
print(info)



{'valid': ['click[back to search]', 'click[next >]', 'click[b09mm9ncqq]', 'click[b09g2f3hq2]', 'click[b09cgnh5xt]', 'click[b09qmj5kqh]', 'click[b09llrswf4]', 'click[b004rk7i2w]', 'click[b096mm7xm4]', 'click[b07cktxs1j]', 'click[b09s2z2gmj]', 'click[b08zkdqpz3]'], 'goal': 'Instruction: i need a long clip-in hair extension which is natural looking, and price lower than 30.00 dollars', 'score': 0.0, 'estimate_score': 0.0, 'prev_ob': 'WebShop\nInstruction: \ni need a long clip-in hair extension which is natural looking, and price lower than 30.00 dollars\n[button] Search [button_]\n', 'desc': '', 'feat': ''}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [20]:
action = "search[long clip-in hair extension natural looking]"
obs, reward, done, info = env.step(action)
print(obs)
print(info)

Instruction:
i need a long clip-in hair extension which is natural looking, and price lower than 30.00 dollars
[button] Back to Search [button_]
Page 1 (Total results: 50)
[button] Next > [button_]

[button] B09MM9NCQQ [button_]
Simayixx Christmas Gift Wigs Hair Extensions 24 Inches Synthetic Long Natural Black Thick Clip Straight Hair Stylish Fasionable Fabulous Hair Piece 6 Pieces Popular Hairstyle(E)
$7.83

[button] B09G2F3HQ2 [button_]
Lurrose 6pcs Girls Hair Bow Ribbon Barrettes Hair Bow Ribbon Clip Wig Hair Bow Clips with Alligator Clip and Hair Extensions Hair Accessory for Toddler Girls Children
$11.99

[button] B09CGNH5XT [button_]
Dual Magnetic Lashes, Magnets False Eyelashes, Soft 3D No Glue Fake Lashes Extension with Tweezers, Natural Look Eyelashes Set with 6 Pieces / 3 Pairs /Mini HD Makeup Mirror
$9.99

[button] B09QMJ5KQH [button_]
Messy Bun Scrunchie Hair Bun Extensions Wavy Curly Messy Donut Hairpieces Synthetic Donut Updo Hair Pieces for Women and Girls (black brown)

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [21]:
action = "click[item - simayixx christmas gift wigs hair extensions 24 inches synthetic long natural black thick clip straight hair stylish fasionable fabulous hair piece 6 pieces popular hairstyle(e)]"
obs, reward, done, info = env.step(action)
print(obs)
print(info)
print(type(info))

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)


Instruction:
i need a long clip-in hair extension which is natural looking, and price lower than 30.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
color
  [button] a [button_]
  [button] b [button_]
  [button] c [button_]
  [button] d [button_]
  [button] e [button_]
  [button] f [button_]
  [button] g [button_]
  [button] h [button_]
  [button] i [button_]
  [button] j [button_]
Simayixx Christmas Gift Wigs Hair Extensions 24 Inches Synthetic Long Natural Black Thick Clip Straight Hair Stylish Fasionable Fabulous Hair Piece 6 Pieces Popular Hairstyle(E)
Price: $7.83
Rating: N.A.
[button] Description [button_]
[button] Features [button_]
[button] Reviews [button_]
[button] Buy Now [button_]

{'valid': ['click[back to search]', 'click[< prev]', 'click[description]', 'click[features]', 'click[reviews]', 'click[buy now]', 'click[a]', 'click[b]', 'click[c]', 'click[d]', 'click[e]', 'click[f]', 'click[g]', 'click[h]', 'click[i]', 'click[j]'], 'goal': 'Instruction: i 

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find(